In [3]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Lambda
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam


import random

print(tf.config.list_physical_devices('GPU'))
df = pd.read_csv("prep_data/image_data.csv")
df.head()

[]


,filename,scalar_x,scalar_y,s_button
0,7000.jpg,0,60,0
1,7001.jpg,0,87,0
2,7002.jpg,0,94,0
3,7003.jpg,0,105,0
4,7004.jpg,0,110,0


In [4]:
def create_input():
    X = []
    y = []
    df = pd.read_csv("prep_data/image_data.csv")
    shuffled_df = df.sample(len(df))
    # 
    print(shuffled_df.head)  # Shuffled DataFrame
    s = 0
    for entry in shuffled_df.values:
        s += 1
        # print(s)
        entry_img = cv2.imread(f"prep_data/{entry[0]}")

        prepared_y = np.delete(entry,0)

        X.append(entry_img)
        y.append(prepared_y)
    y = np.asarray(y).astype(np.float32)

    return X, y


In [5]:
X, y = create_input()



<bound method NDFrame.head of      filename  scalar_x  scalar_y  s_button
120  7120.jpg        89       204       100
123  7123.jpg        93       179       100
65   7065.jpg       350        13         0
156  7156.jpg       125       159       100
102  7102.jpg       369       264         0
..        ...       ...       ...       ...
125  7125.jpg        95       207       100
169  7169.jpg       129       160       100
84   7084.jpg        36       276         0
200  7200.jpg       146       213       100
0    7000.jpg         0        60         0

[222 rows x 4 columns]>


In [6]:
def compile_model():
    image_width, image_height, channels = 640, 360, 3  # Assuming RGB images

    model = Sequential()
    model.add(Input(shape=(image_height, image_width, channels)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(3, activation='linear'))

    model.compile(loss='mean_squared_error', optimizer=Adam())

    return model



In [7]:
def train_model(model,X,y, epochs_req):

    train_split = int(len(y) * 0.8)
    X_train, X_test = np.array(X[:train_split - 1]), np.array(X[train_split:])
    y_train, y_test = y[:train_split - 1], y[train_split:]
    model.fit(X_train, y_train, epochs=(epochs_req), validation_data=(X_test, y_test))

    for i in range(5):
        tbp = np.asarray(X_test[i])
        tbp = np.expand_dims(tbp, axis=0)
        predictions = model.predict(tbp)
        print("pred", predictions, "real", y_test[i])


    return model

In [8]:
# model = compile_model()
# model = train_model(model, X, y, 2)
# model.save('cemetery.h5')

In [ ]:
from keras.src.saving import load_model
import os

file_path = r'cemetery.h5'

if not os.path.exists(file_path):
    print(f"File does not exist: {file_path}")
else:
    try:
        
        model_rap = load_model(file_path)
        model_rap.summary()

        # Compile the model (if necessary)
        model_rap.compile(loss='mean_squared_error', optimizer=Adam())

        train_model(model_rap, X, y, 20)
        model_rap.save('cemetery.h5')

    except Exception as e:
        print(f"An error occurred: {e}")

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ max_pooling2d_12 (MaxPooling2D) │ (None, 180, 320, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 178, 318, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 89, 159, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 905664)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 64)             │    57,962,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 57,972,869 (221.15 MB)

 Trainable params: 57,972,867 (221.15 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

Epoch 1/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - loss: 7940.4458 - val_loss: 5656.1353
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - loss: 5326.1235 - val_loss: 3686.0610
Epoch 3/20
4/6 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - loss: 4873.1011